In [1]:
# import nltk

# # Download required NLTK data (only need to do this once)
# nltk.download('punkt')

# random notes 2-17 on algo improvement:

- "weight" articles parsed based on recency (more recent articles are weighted more heavily)
- news sources miss things social media has --> should hit some of those APIs
- can help daniel by creating targeted sora prompts 
- can help daniel by creating grok prompts for specific images


# Alterations to algo, 2/24
- shorter length per scene, but more scenes
- each scene should be more exciting/dramatic/crazy

# My thoughs on how to do that:
- make script with GROK not GPT-4o --> this will make asset generation downstream much easier

## AI VIDEO EDITOR // AI VIDEO EDITING WORKFLOW IS THE KEY MISSING PIECE TO A FULLY OPTIMIZED PIPELINE

In [1]:
import trendspy
from trendspy import Trends
import os
import datetime
from dotenv import load_dotenv
from news_utils import News
import feedparser
import time
import pickle
from generateAssets import *
load_dotenv()

c:\Users\garre\OneDrive\Desktop\Projects\aiMedia


True

In [2]:
# Make directory for the run
APPDIR = os.getenv("APPDIR")
BASEPATH = os.getenv("BASEPATH")
RUN_DIR = os.path.join(BASEPATH, datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(BASEPATH, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.chdir(APPDIR)

In [3]:
os.environ["RUN_DIR"] = RUN_DIR

In [4]:
def getTrends():
    tr = Trends()
    trends = tr.trending_now(geo='US', hours=24)
    businessFiltered = trends.filter_by_topic('Business and Finance')
    techFiltered = trends.filter_by_topic('Technology')
    lawAndGovFiltered = trends.filter_by_topic('Law and Government')
    filteredTrends = businessFiltered + techFiltered + lawAndGovFiltered
    sortedTrends = sorted(filteredTrends, key=lambda x: x.volume, reverse=True)
    finalTrendList = []
    for trend in sortedTrends:
        finalTrendList.append({
            'keyword': trend.keyword,
            'volume': trend.volume,
            'volume_growth_pct': trend.volume_growth_pct,
            'topics': trend.topics,
            'keywords': trend.trend_keywords,
        })
    return finalTrendList

In [5]:
# Example usage in your getNews function:
def getNews(trendKeyword, num_articles=10, linkNews={}):
    search_term = trendKeyword.replace(' ', '+')
    url = f'https://news.google.com/rss/search?q={search_term}&hl=en-US&gl=US&ceid=US:en'
    feed = feedparser.parse(url)
    valid, invalid = [], []

    for entry in feed.entries[:min(num_articles, len(feed.entries))]:
        if entry.link not in linkNews:
            newNews = News(
                entry.title,
                entry.link,
                entry.published,
            )
            linkNews[entry.link] = newNews
            if newNews.mainText:
                valid.append(newNews)
            else:
                invalid.append(newNews)

    return valid, invalid

In [6]:
def collectArticles(trend, linkNews={}, maxArticles = 25):
    news = set()
    keywords = set(trend['keywords'] + trend['keywords'])
    numArticlesPerKeyword = 1 + (maxArticles // len(keywords)) # promotes diversity across keywords
    for keyword in keywords:
        time.sleep(2) # wait two seconds to avoid rate limiting
        valid, invalid = getNews(keyword, num_articles=numArticlesPerKeyword, linkNews=linkNews)
        news.update(valid)
        if len(news) > maxArticles:
            break
    trend['news'] = list(news)

# 🔄 Collect Trends Or Load From Pickle
---

In [9]:
getTrendInfo = input('Get Trends? (y/n): ')
if getTrendInfo.lower() == 'y':
    finalTrendList = getTrends()
    showTrends = input('Show Trends? (y/n): ')
    if showTrends.lower() == 'y':
        outStr = """"""
        for trend in finalTrendList[:min(100, len(finalTrendList))]:
            outStr += f"**{trend['keyword']}**\n"
            for k, v in trend.items():
                if k != 'keyword':
                    outStr += f"{k} : {v}\n"
        print(outStr)
    saveTrends = input('Save Trends? (y/n): ')
    if saveTrends.lower() == 'y':
        with open('./tmp/finalTrendList.pkl', 'wb') as f:
            pickle.dump(finalTrendList, f)
elif getTrendInfo.lower() == 'n':
    with open('./tmp/finalTrendList.pkl', 'rb') as f:
        finalTrendList = pickle.load(f)
else:
    print('Invalid input, run again')
    exit()

**9070 xt**
volume : 50000
volume_growth_pct : 300
topics : [18]
keywords : ['9070 xt', '9070xt', '9070 xt release date', 'rx 9070 xt', 'rx 9070 xt release date', 'amd 9070 xt', 'amd radeon rx 9070 xt release date']
**hayden haynes**
volume : 50000
volume_growth_pct : 1000
topics : [10, 14, 1]
keywords : ['hayden haynes', 'mike johnson dui', 'mike johnson chief of staff', 'mike johnson arrested', 'hayden haynes mike johnson']
**stock markets**
volume : 20000
volume_growth_pct : 600
topics : [3]
keywords : ['stock markets', 'dow jones stock', 'stock market dow', 'djia futures', 's&p 500 futures', 'stock futures today', 'djia', 'futures', 'djia today', 's&p 500 today', 'tariff relief', 'market futures', 'yahoo finance', 's&p futures', 'markets today']
**mexico closes border**
volume : 20000
volume_growth_pct : 400
topics : [10, 14]
keywords : ['mexico closes border', 'mexico border', 'jd']
**michelle wu**
volume : 20000
volume_growth_pct : 200
topics : [10, 14]
keywords : ['michelle wu',

# 🎯 Select Trend and Collect Articles
---

In [10]:
keyword = input('Enter trend keyword: ')
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]
linkNews = {}
collectArticles(trend, linkNews=linkNews, maxArticles = 40)
pickle.dump(finalTrendList, open('./tmp/finalTrendList.pkl', 'wb'))
print('Trends info scraped, trend selected and articles collected. All information saved.')

URL https://news.google.com/rss/articles/CBMikwFBVV95cUxOZV9BRW0wWTEzRW1FWThwcE9meEhyQkpNdngzUmJpSS1aT2NVUHl6aXA2WTc0Q1lFVDdzLU5DZ2hIQXJYYTEwcU9hR2tVSFFIWElscTNZTjlNTVFRa3NhSzRhWXd5TFNPdFpiY2diZWRzNlEtNVVZWFlKdDlNWk8ydmJtb0FKZWlCVGlRQzdYenVISGM?oc=5
URL https://news.google.com/rss/articles/CBMimgFBVV95cUxQUVhhckhyZFV0OVJjLW5tMXdnMDg0a0hueXZJWGZXVTRIMVRZdDlab1V1Q1JSMzd6YjQ5QWE0dmpfNkNadlprN0RkMEhsQ0JGWjRla3hpY1ExTmFveGY0YXlnVm5xQ2pUeDN2YjNRemNINkNyQTdyYXBhVjVJZFYzbmhRYVdrUllzZ2dvOWFnUmcxTUkyb1RkUTZ30gGfAUFVX3lxTE44SWhLXzNaTGdUQjE2WlJGaEhGSWJOWnVaWk5sZklDMi01VmdUVVF0ekNuSy1xakgwVVFla3hmbXp3SUw2R2xEdkpQTF9lcmdodkRBU2RleEdyMUVuVk0yaS1fZjNBMTgxa1k1S3BKdkEzSUVFUTZuSTJ4NXVVWlQ5RjZKVUZ5dWF4ZFlhZ3VtYVlFNWdwU3RaUlVOcS01SQ?oc=5
URL https://news.google.com/rss/articles/CBMiswFBVV95cUxOc0xBQmtSZFdFaEsyNk5FX2ZjaUVnclFzWnEwenF2b0NVQ2V0WG5MSUVLcmlIXzc3WTBTbVpuMEN5VEpjdE9SWm1Hd0ZMc3NFSkt2YmhncURYMktKN2NxRUpNZHZCaGIxZHdwYXJMNjNvOV9JNXBxUlpiYnZuRDJJQ1lUc0pPaXdTQUxUdXZmai1HTF9vOV9JUXBIel9wenRSTnlCbGNubEdD

# 🎨 Generate Assets For Selected Trend
---

In [11]:
config = {'news_report': True,
        'script': True,
        'narration': True,
        'voice': False,
        'sora_prompts': True}

In [12]:
print(ELEVENLABS_API_KEY)

sk_f96d619fc51235180796190a025b96954210e5b25af0d67d


In [13]:
generate_assets(trend, openai_api_key=OPENAI_API_KEY, elevenlabs_api_key=ELEVENLABS_API_KEY, voice_id=VOICE_ID, run_dir=RUN_DIR, config=config)

C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-05-21-37-19\department_of_education topic_dir
C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-05-21-37-19\department_of_education\sora_prompts sora_prompt_dir
32 validArticles


{'paths': {'script': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-05-21-37-19\\department_of_education\\script.txt',
  'audio': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-05-21-37-19\\department_of_education\\voiceover.mp3',
  'video': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-05-21-37-19\\department_of_education\\video.mp4',
  'sora_prompts': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-05-21-37-19\\department_of_education\\sora_prompts',
  'news_report': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-05-21-37-19\\department_of_education\\news_report.txt'}}

In [22]:
os.getenv("RUN_DIR")

'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-04-21-53-26'

In [18]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

In [19]:
!echo python -m wormhole send {outName}.zip | clip && start cmd